In [1]:
from analysis import analyze_repository

repo_path = "/home/ssimon/projects/test-config-repo"
commit_data = analyze_repository(repo_path=repo_path, project_name="test-config-repo", get_diff=True)

config_commit_data = commit_data["config_commit_data"]

for commit in config_commit_data:
    print("Commit hash:", commit["commit_hash"])
    print("Commit message:", commit["commit_mgs"])
    network_data = commit["network_data"]
    if network_data:
        for x in network_data["config_files_data"]:
            print("Added pairs: ", x["added_pairs"])
            print("Removed pairs: ", x["removed_pairs"])
            print("Modified pairs: ", x["modified_pairs"])
    print("\n\n")

Number of commits: 6


Processing: 100%|██████████| 6/6 [00:00<00:00, 57.62it/s]

Len commit data: 6, 1.0
Elapsed time: 0.116706 seconds
Commit hash: 157a941cd30350e745bb28db886fd483c43451ce
Commit message: Initial commit

Added pairs:  [{'artifact': 'src/Dockerfile', 'option': 'FROM', 'value': 'maven:3.8.6-eclipse-temurin-17 AS build', 'line': 2, 'type': 'IMAGE'}, {'artifact': 'src/Dockerfile', 'option': 'WORKDIR', 'value': '/app', 'line': 3, 'type': 'PATH'}, {'artifact': 'src/Dockerfile', 'option': 'COPY', 'value': 'pom.xml .', 'line': 6, 'type': 'UNKNOWN'}, {'artifact': 'src/Dockerfile', 'option': 'RUN', 'value': 'mvn dependency:go-offline -B', 'line': 7, 'type': 'COMMAND'}, {'artifact': 'src/Dockerfile', 'option': 'COPY', 'value': 'src ./src', 'line': 10, 'type': 'UNKNOWN'}, {'artifact': 'src/Dockerfile', 'option': 'RUN', 'value': 'mvn package -DskipTests', 'line': 11, 'type': 'COMMAND'}, {'artifact': 'src/Dockerfile', 'option': 'FROM', 'value': 'eclipse-temurin:17-jdk-alpine', 'line': 14, 'type': 'IMAGE'}, {'artifact': 'src/Dockerfile', 'option': 'WORKDIR', 'va